In [ ]:
# Import libraies
import pandas as pd
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
import mysql.connector
from datetime import datetime

##Connect to the mysql DB
try:
        connection = mysql.connector.connect(host='stockdb.cnu60g6iqlqd.eu-north-1.rds.amazonaws.com', user='admin', password='4hRmqNgg32DFbrg3WHfm', database='stockDB', port=3306)
except:
        print("Connection failed")

cursor = connection.cursor()

# Define consumer object
consumer = KafkaConsumer(
    'stock',
    bootstrap_servers=['113.60.70.157:9092'], 
    value_deserializer=lambda x: loads(x.decode('utf-8')))

# Capture msg by msg
for message in consumer:
   # print(message.value)
    
    #Transform data
    msg = message.value[0]
    Index = msg['Index']
    Date = datetime.strptime(msg['Date'], '%Y-%m-%d').date()
    Open = msg['Open']
    High = msg['High']
    Low = msg['Low']
    Close = msg['Close']
    Adj_Close = msg['Adj Close']
    Volume = msg['Volume']
    CloseUSD = msg['CloseUSD']
    
    # Insert query
    sqlQuery = "insert into stockdata(IndexName, Date, Open, High, Low, Close, Adj_Close, Volume, CloseUSD) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    # Insert the data
    result = cursor.execute(sqlQuery,(Index,Date,Open,High,Low,Close,Adj_Close,Volume,CloseUSD))
    # Commit the chages
    connection.commit()

#Close the mysql connector
connection.close()